In [1]:
import json
import random
import string

import tornado.httpclient
import tornado.gen
import tornado.options
import tornado.web
import tornado.websocket

import numpy as np

import h5py

In [2]:
def parse_payload(data):
    try:
        body = None
        frame = json.loads(data)
        if isinstance(frame, list) and len(frame) > 0:
            frame = frame[0]
            body = json.loads(frame.get('body', "{}"))

        return frame, body

    except Exception as ex:
        print(ex)

    return None, None

In [3]:
@tornado.gen.coroutine
def connect_and_read_websocket():

    url = "wss://www.polenes.com/sockjs/490/j6ieo28e/websocket"
    try:
        print("connecting to: %s", url)
        w = yield tornado.websocket.websocket_connect(url, connect_timeout=5)
        print("connected to %s, waiting for messages", url)
    except Exception as ex:
        print("couldn't connect, err: %s", ex)
        return

    f = h5py.File('pollen.h5', 'w');

    day = np.zeros((1, 24), dtype=np.int32)
    data = 'null'

    while True:
        payload = yield w.read_message()
        if payload is None:
            print("uh oh, we got disconnected")
            table.flush();
            return

        if payload[0] == "o":
            msg = json.dumps(['{"msg":"connect", "version":"1", "support":["1","pre2","pre1"]}'])
            w.write_message(msg.encode('utf8'))
        elif payload[0] == 'a':
            frame = json.loads(payload[1:])
            body = json.loads(frame[0])

            if 'msg' in body:
                if body['msg'] == 'connected':
                    stringmsg = '{{"msg":"sub","id":"die7LBp7mEbzW3ffQ","name":"polenes2015","params":[{{"selector":{{"$and":[{{"fecha":{{"$gte":19750101,"$lte":20201231}}}},{{"idEstacion":29}}]}},"options":{{"sort":{{"fecha":1}}}},"jump":1}}]}}'.format();
                    print(stringmsg)
                    msg = json.dumps([stringmsg])
                    w.write_message(msg.encode('utf8'));
                elif body['msg'] == 'ping':
                    print('pong')
                    msg = json.dumps(['{"msg": "pong"}'])
                    w.write_message(msg.encode('utf8'));
                elif body['msg'] == 'ready':
                    print(data.shape)
                    f.create_dataset('madrid_subiza', data=data)
                    return
                elif body['msg'] == 'added':
                    day[0][0] = body['fields']['fecha'];

                    for i in range(1, 23):
                        if str(i) in body['fields']:
                            day[0][i] = body['fields'][str(i)];
                        else:
                            day[0][i] = 0;

                    if data == 'null':
                        data = day
                    else: 
                        data = np.append(data, day, axis=0);

In [4]:
tornado.ioloop.IOLoop.instance().run_sync(connect_and_read_websocket)

RuntimeError: This event loop is already running

connecting to: %s wss://www.polenes.com/sockjs/490/j6ieo28e/websocket
connected to %s, waiting for messages wss://www.polenes.com/sockjs/490/j6ieo28e/websocket
{"msg":"sub","id":"die7LBp7mEbzW3ffQ","name":"polenes2015","params":[{"selector":{"$and":[{"fecha":{"$gte":19750101,"$lte":20201231}},{"idEstacion":29}]},"options":{"sort":{"fecha":1}},"jump":1}]}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


pong


Da un error pero funciona xd